In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

In [2]:
# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


In [3]:
# define a model
model = Sequential([
    Flatten(input_shape = (28,28)),
    Dense(128, activation = 'relu'),
    Dense(10)
])

c:\Users\gunay\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# define loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

In [5]:
# implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.349669933319092
Epoch 1 Step 200: Loss = 0.38923656940460205
Epoch 1 Step 400: Loss = 0.18076735734939575
Epoch 1 Step 600: Loss = 0.19310754537582397
Epoch 1 Step 800: Loss = 0.1458130031824112
Epoch 1 Step 1000: Loss = 0.4679909348487854
Epoch 1 Step 1200: Loss = 0.1573670506477356
Epoch 1 Step 1400: Loss = 0.2595573663711548
Epoch 1 Step 1600: Loss = 0.19441808760166168
Epoch 1 Step 1800: Loss = 0.20664526522159576
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07567359507083893
Epoch 2 Step 200: Loss = 0.11164087057113647
Epoch 2 Step 400: Loss = 0.11896685510873795
Epoch 2 Step 600: Loss = 0.05577671527862549
Epoch 2 Step 800: Loss = 0.09995268285274506
Epoch 2 Step 1000: Loss = 0.26084694266319275
Epoch 2 Step 1200: Loss = 0.09275107830762863
Epoch 2 Step 1400: Loss = 0.196408212184906
Epoch 2 Step 1600: Loss = 0.17089885473251343
Epoch 2 Step 1800: Loss = 0.14739060401916504


In [6]:
# enhance the custom training loop by adding an accuracy metric to monitor model performance

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


In [7]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 0.034793607890605927 Accuracy = 1.0
Epoch 1 Step 200: Loss = 0.03796670585870743 Accuracy = 0.9749689102172852
Epoch 1 Step 400: Loss = 0.12114369124174118 Accuracy = 0.9721789360046387
Epoch 1 Step 600: Loss = 0.024428755044937134 Accuracy = 0.9737417101860046
Epoch 1 Step 800: Loss = 0.06340333819389343 Accuracy = 0.9740168452262878
Epoch 1 Step 1000: Loss = 0.16261179745197296 Accuracy = 0.9745879173278809
Epoch 1 Step 1200: Loss = 0.07859534025192261 Accuracy = 0.9750728607177734
Epoch 1 Step 1400: Loss = 0.10269129276275635 Accuracy = 0.9755085706710815
Epoch 1 Step 1600: Loss = 0.12958714365959167 Accuracy = 0.9750156402587891
Epoch 1 Step 1800: Loss = 0.08711864799261093 Accuracy = 0.9754476547241211
Start of epoch 2
Epoch 2 Step 0: Loss = 0.015432622283697128 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.015945684164762497 Accuracy = 0.9811878204345703
Epoch 2 Step 400: Loss = 0.08605445176362991 Accuracy = 0.9806733131408691
Epoch 2 Step 600

In [8]:
# Custom callback for advanced logging
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [9]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 0.009429750964045525 Accuracy = 1.0
Epoch 1 Step 200: Loss = 0.013684853911399841 Accuracy = 0.9942474961280823
Epoch 1 Step 400: Loss = 0.03636747971177101 Accuracy = 0.9945448637008667
Epoch 1 Step 600: Loss = 0.03305281698703766 Accuracy = 0.9945923686027527
Epoch 1 Step 800: Loss = 0.03783285990357399 Accuracy = 0.9946160912513733
Epoch 1 Step 1000: Loss = 0.04316720366477966 Accuracy = 0.9945366978645325
Epoch 1 Step 1200: Loss = 0.014143750071525574 Accuracy = 0.9945878386497498
Epoch 1 Step 1400: Loss = 0.013795854523777962 Accuracy = 0.9945128560066223
Epoch 1 Step 1600: Loss = 0.018518628552556038 Accuracy = 0.994573712348938
Epoch 1 Step 1800: Loss = 0.012339422479271889 Accuracy = 0.9945690035820007
End of epoch 1, loss: 0.002918898593634367, accuracy: 0.9946833252906799
Start of epoch 2
Epoch 2 Step 0: Loss = 0.016985910013318062 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.0026502422988414764 Accuracy = 0.9937810897827148
Epoch 2 Step 4

In [10]:
# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In [11]:
# define the output
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In [12]:
# create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [13]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# train the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4890 - loss: 0.7022 
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5110 - loss: 0.6940
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5210 - loss: 0.6913 
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5280 - loss: 0.6903
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5650 - loss: 0.6873
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5570 - loss: 0.6866 
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5650 - loss: 0.6838
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5870 - loss: 0.6824 
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5470 - loss: 0.6832
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5860 - loss: 0.6800


In [15]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5000 - loss: 0.6902 
Test loss: 0.6901620626449585
Test accuracy: 0.5
